- 합성곱 계층

- 윈도(window / filter)라는 가중치의 유닛이 이전 계츠으이 모여 있는 일정한 수의 유닛에 연결된다.

- Convolution Layer
- Fully Connected가 아니라 일부만 연결한 것
- Fully Connected = Dense

- 활성함수 (Activation Function)
- sigmoid
- -1에서 1 사이의 값을 출력 = tanh:
    - 음수값이 유호하면 sigmoid를 쓰지 말고 tanh를 사용하라
- ReLu:
    - sigmoid 함수는 미분이 반복될수록 0에 가까워 진자
    - 따라서 층이 깊어질수록 sigmoid를 사용할 경우 끝에 있는 것은 영향력이 거의 0에 수렴한다.
    - 이에 따라 상한선을 제거 -> ReLu
    - ReLu를 쓰게 되면 마이너스를 다룰 수 없다.
- Leaky ReLu:
    - ReLu에서 마이너스를 유효하게 한 것.
    - 마이너스 구간의 기울기는 가파르지 않다.
- Maxout:
    - 둘 중에 무조건 큰 것을 도출
- ELU


- softmax:
    - class가 여러개 일 때 결과를 각각의 확률을 따져 제일 높은 확률로 나온 것이 정답인 것으로 도출

In [ ]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
tf.set_random_seed(777)

x_data = [[1,2,1,1],
         [2,1,3,2],
         [3,1,3,4],
         [4,1,5,5],
         [1,7,5,5],
         [1,2,5,6],
         [1,6,6,6],
         [1,7,7,7]]

y_data = [[0,0,1],
         [0,0,1],
         [0,0,1],
         [0,1,0],
         [0,1,0],
         [0,1,0],
         [1,0,0],
         [1,0,0]]

In [ ]:
X = tf.placeholder('float',[None,4])
Y = tf.placeholder('float',[None,3]) #다중 클래스
nb_classes = 3
W = tf.Variable(tf.random_normal([4,nb_classes]), name = 'weight') #Y가 (n,3)이므로 (4,3)
b = tf.Variable(tf.random_normal([nb_classes]), name = 'bias')

#softmax = exp(logits) / reduce_sum(exp(logits),dim)
hypothesis = tf.nn.softmax(tf.matmul(X, W) + b)

cost = tf.reduce_mean(-tf.reduce_sum(Y * tf.log(hypothesis), axis = 1))

optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.1).minimize(cost)



with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    for step in range(2001):
        sess.run(optimizer, feed_dict = {X:x_data, Y:y_data})
        if step % 200 ==0:
            print(step, sess.run(cost, feed_dict = {X:x_data, Y:y_data}))
    a = sess.run(hypothesis, feed_dict = {X : [[1,11,7,9]]})
    print(a,sess.run(tf.argmax(a,1)))

- 역전파 알고리즘:
    - 말 그대로 거꾸로 거슬러 올라가 가중치를 수정하는 것, 각 과정마다 편미분을 하여
    - 영향을 끼치는 정도를 알아낸다
- 드롭아웃:
    - 일반적으로 Fully Connected로 되어 있었다.
    - -> 이것을 랜덤하게 연결하여 보자(Dropout)
    - keras나 tensorflow에서 dropout의 파라미터가 살짝씩 다르다 (In Out)
    - batch 때마다 적용이 된다.
    - 검증을 할 떄는 Dropout =0 , 즉 적용이 되어서는 안된다.


In [ ]:
Y = [[1],[3],[0]]
Y_one_hot = tf.one_hot(Y,4)
print(Y_one_hot)
#Y_one_hot.shape = (3,1,4)
#네개 짜리 데이터의 한 묶음이 세개가 있다.


In [ ]:
data = np.loadtxt('C:/Users/edu/venv/Scripts/data-04-zoo.csv', delimiter = ',',skiprows = 1, dtype = np.float32)

In [ ]:
data.shape

In [ ]:
x_data = data[:,0:-1] 
y_data = data[:,[-1]] 

nb_classes = 7
X = tf.placeholder(tf.float32, shape = (None,16))
W = tf.Variable(tf.random_normal([16,nb_classes]), name = 'weight')
b = tf.Variable(tf.random_normal([nb_classes]), name = 'bias')
Y = tf.placeholder(tf.int32, shape = (None,1))
Y_one_hot = tf.one_hot(Y,nb_classes)
Y_one_hot = tf.reshape(Y_one_hot, [-1,nb_classes])
print('reshape : ', Y_one_hot)

In [ ]:
print(x_data.shape)
print(y_data.shape)

In [ ]:
logits = tf.matmul(X,W) + b

hypothesis = tf.nn.softmax(logits)

cost_i = tf.nn.softmax_cross_entropy_with_logits(logits = logits, labels = Y_one_hot)

cost = tf.reduce_mean(cost_i)

optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.1).minimize(cost)

prediction = tf.argmax(hypothesis,1)
correct_prediction = tf.equal(prediction, tf.argmax(Y_one_hot,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))


with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    for step in range(2001):
        sess.run(optimizer, feed_dict = {X:x_data, Y:y_data})
        if step % 200 ==0:
            loss, acc = sess.run([cost, accuracy],feed_dict = {X : x_data, Y:y_data})
            print('step : {:5}\tLoss : {:.3f}\tAcc : {: .2%}'.format(step, loss, acc))
    pred = sess.run(predcition, feed_dict = {X : x_data})
    for p,y in zip(pred, y_data.flatten()):
        print("[{}] Prediction : {} True Y : {}".format(p == int(y), p, int(y)))